In [87]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import RandomizedSearchCV

In [88]:
#import the data
train_x = pd.read_csv("train_dummy_upsampled_final.csv")
# train_y = pd.read_csv("TrainingSetLabels.csv")
test_x = pd.read_csv("test_dummy_final.csv")
data = pd.read_csv("names.csv")

In [89]:
#drop permit 
train_x = train_x.drop(columns=['public_meeting','permit'])
test_x = test_x.drop(columns=['public_meeting','permit'])

In [90]:
# # set ID as index for train x and y 
train_x = train_x.set_index(['id'])
# train_y = train_y.set_index(['id'])

In [91]:
#create x/y split for trianing 
train_y = pd.DataFrame(train_x['status_group'])
train_x = train_x.drop(['status_group'], axis = 1)

In [92]:
#save test id and drop 
test_id = pd.DataFrame(test_x['id'])
test_x = test_x.drop(['id'], axis = 1)

In [93]:
# #convert output status group to 1,2,3 
 
# replace_map = {"non functional":3, 
#               "functional needs repair":2,
#               "functional":1}

# train_y['status_group code'] = train_y['status_group'].replace(replace_map)

In [97]:
#split data just for testing 
X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y, train_size=0.6, random_state=42)
#Identify cat features for model
categorical_features_indices = np.where(train_x.dtypes != np.float)[0]

In [98]:
# knn = KNeighborsClassifier(n_neighbors = 10)

#grid search and RandomizedSearchCV parapeters 
k_range = list(range(1, 31))
weight_options = ['uniform', 'distance']


# use grid search to find best parameters 
param_grid = dict(n_neighbors=k_range, weights=weight_options)
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', n_jobs = 40)
# fit the grid with data
grid.fit(X_train, Y_train.values.ravel())
print(grid.best_score_)
print(grid.best_params_)

# #RandomizedSearchCV to reduce complexity 
# param_dist = dict(n_neighbors=k_range, weights=weight_options)

# rand = RandomizedSearchCV(knn, param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=5)
# # fit the grid with data
# rand.fit(X_train, Y_train.values.ravel())
# print(rand.best_score_)
# print(rand.best_params_)


0.7747735592492011
{'n_neighbors': 2, 'weights': 'distance'}


In [96]:
# training a KNN classifier 

knn = KNeighborsClassifier(n_neighbors = 1 , weights = 'uniform')
knn.fit(X_train, Y_train.values.ravel()) 
# accuracy on X_test 
accuracy = knn.score(X_test, Y_test) 
print(accuracy)
  
# creating a confusion matrix 
knn_predictions = knn.predict(X_test)  
cm = confusion_matrix(Y_test, knn_predictions) 

0.7941034649032169


In [81]:
#full model 
knn = KNeighborsClassifier(n_neighbors = 1 , weights = 'uniform').fit(train_x, train_y.values.ravel()) 

In [82]:
#submision
preds_class_full = knn.predict(test_x) 


In [83]:
preds_class_full = pd.DataFrame(preds_class_full)
preds_class_full.columns = ['pre']
preds_class_full = preds_class_full['pre']

In [84]:
submission = pd.concat([test_id, preds_class_full], axis=1)
submission.columns = ['id', 'status_group']
replace_map2 = {2:"non functional", 
              1:"functional needs repair",
              0:"functional"}

submission['status_group'] = submission['status_group'].replace(replace_map2)
submission.to_csv("submision_knn.csv", index=False)